In [1]:
import functions_framework
import pandas as pd
import requests 
import sqlalchemy
from datetime import datetime
from pytz import timezone

@functions_framework.http
def update_weather(request):
    connection_string = connection()
    cities = cities_list(connection_string)
    weather_df = fetch_weather_data(cities)
    weather_info = merged(weather_df, connection_string)
    weather_to_sql(weather_info, connection_string)
    
    return 'Data successfully added'

def connection():
    schema = "gans"
    host = "My_IP"
    user = "root"
    password = "My_pass"
    port = 3306
    db = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return db

def cities_list(connection_string):
    cities_df = pd.read_sql('cities_info', con=connection_string)
    return list(cities_df['city_name'])

def fetch_weather_data(cities):
    try:     
        berlin_timezone = timezone('Europe/Berlin')
        API_key = My_API_pass  
        weather_items = []

        for city in cities:
            country = "DE"
            geo_url = f"http://api.openweathermap.org/geo/1.0/direct?"\
                      f"q={city},{country}&limit=5&appid={API_key}"
            geo_response = requests.get(geo_url)
            geo_response.raise_for_status()
            geo_json = geo_response.json()

            if not geo_json:
                continue

            latitude = geo_json[0]["lat"]
            longitude = geo_json[0]["lon"]

            weather_url = f"https://api.openweathermap.org/data/2.5/"\
                          f"forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric"
            weather_response = requests.get(weather_url)
            weather_response.raise_for_status()
            weather_json = weather_response.json()

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in weather_json["list"]:
                weather_item = {
                    "city_name": city,
                    "forecast_time": item.get("dt_txt", None),
                    "temperature": item["main"].get("temp", None),
                    "feels_like": item["main"].get("feels_like", None),
                    "outlook": item["weather"][0].get("main", None),
                    "desc_of_outlook": item["weather"][0].get("description", None),
                    "wind_speed": item["wind"].get("speed", None),
                    "rain_prob": item.get("pop", 0),
                    "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                    "retrievd_info_date": retrieval_time
                }
                weather_items.append(weather_item)

        weather_df = pd.DataFrame(weather_items)
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        raise
    except Exception as e:
        print(f"Error: {e}")
        raise

    return weather_df

def merged(weather_df, connection_string):
    cities_df = pd.read_sql('cities_info', con=connection_string)
    city_weather_merged = weather_df.merge(cities_df, on='city_name', how='left')
    
    city_weather_merged = city_weather_merged[['city_id'] + [col for col in city_weather_merged.columns if col != 'city_id']]
    weather_info = city_weather_merged.drop(columns=['city_name', 'city_latitude', 'city_longitude', 'country_code', 'population'], axis=1)

    return weather_info

def weather_to_sql(weather_info, connection_string):
    weather_info.to_sql(name='weather_info', con=connection_string, if_exists='append', index=False)
